In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import os

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [3]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [4]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [5]:
n_inputs = 28*28  # Adequar para o XOR
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [6]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

In [7]:
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1",
                              activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2",
                              activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    y_proba = tf.nn.softmax(logits)

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


In [9]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [10]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [11]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    accuracy_summary = tf.summary.scalar('accuracy', accuracy)

In [12]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [13]:
from datetime import datetime

def log_dir(prefix=""):
    now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    root_logdir = "tf_logs"
    if prefix:
        prefix += "-"
    name = prefix + "run-" + now
    return "{}/{}/".format(root_logdir, name)

In [14]:
m, n = X_train.shape
logdir = log_dir("mnist_dnn")
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [16]:
n_epochs = 10001
batch_size = 50
n_batches = int(np.ceil(m / batch_size))

final_model_path = "./my_deep_mnist_model"

best_loss = np.infty
epochs_without_progress = 0
max_epochs_without_progress = 50

with tf.Session() as sess:
    start_epoch = 0
    sess.run(init)

    for epoch in range(start_epoch, n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val, loss_val, accuracy_summary_str, loss_summary_str = sess.run([accuracy, loss, accuracy_summary, loss_summary], feed_dict={X: X_valid, y: y_valid})
        file_writer.add_summary(accuracy_summary_str, epoch)
        file_writer.add_summary(loss_summary_str, epoch)
        if epoch % 5 == 0:
            print("Epoch:", epoch,
                  "\tValidation accuracy: {:.3f}%".format(accuracy_val * 100),
                  "\tLoss: {:.5f}".format(loss_val))
            if loss_val < best_loss:
                saver.save(sess, final_model_path)
                best_loss = loss_val
            else:
                epochs_without_progress += 5
                if epochs_without_progress > max_epochs_without_progress:
                    print("Early stopping")
                    break

Epoch: 0 	Validation accuracy: 90.260% 	Loss: 0.35365
Epoch: 5 	Validation accuracy: 95.180% 	Loss: 0.17470
Epoch: 10 	Validation accuracy: 96.460% 	Loss: 0.12657
Epoch: 15 	Validation accuracy: 97.240% 	Loss: 0.10385
Epoch: 20 	Validation accuracy: 97.360% 	Loss: 0.09243
Epoch: 25 	Validation accuracy: 97.720% 	Loss: 0.08146
Epoch: 30 	Validation accuracy: 97.800% 	Loss: 0.07617
Epoch: 35 	Validation accuracy: 97.960% 	Loss: 0.07270
Epoch: 40 	Validation accuracy: 97.980% 	Loss: 0.06917
Epoch: 45 	Validation accuracy: 98.080% 	Loss: 0.06748
Epoch: 50 	Validation accuracy: 98.080% 	Loss: 0.06643
Epoch: 55 	Validation accuracy: 98.100% 	Loss: 0.06571
Epoch: 60 	Validation accuracy: 98.120% 	Loss: 0.06674
Epoch: 65 	Validation accuracy: 98.080% 	Loss: 0.06741
Epoch: 70 	Validation accuracy: 98.120% 	Loss: 0.06728
Epoch: 75 	Validation accuracy: 98.100% 	Loss: 0.06644
Epoch: 80 	Validation accuracy: 98.180% 	Loss: 0.06568
Epoch: 85 	Validation accuracy: 98.180% 	Loss: 0.06731
Epoch: 90 	V